In [1]:
#@title HERE
import numpy as np
import pandas as pd
import requests
from scipy.optimize import minimize, differential_evolution
from tqdm import tqdm
from astropy import constants as const
from astropy import units as u

# Physical constants with correct units
G = const.G.to(u.kpc * (u.km/u.s)**2 / u.M_sun).value  # (kpc*(km/s)^2/M_sun)
a0 = 1.2e-10 * u.m/u.s**2  # MOND constant
a0 = a0.to((u.km/u.s)**2 / u.kpc).value  # (km/s)^2/kpc

# Correct handling of the Hubble constant
H0 = 70.0 * u.km / u.s / u.Mpc  # 70 km/s/Mpc
H0_kpc = H0.to(u.km / u.s / u.kpc).value  # Converting to (km/s)/kpc, will be ~0.07

# Function to check converted units
def check_units():
    v200_test = 200.0  # km/s, typical value
    r200_test = v200_test / (10 * H0_kpc)  # kpc

    print(f"Checking units of measurement:")
    print(f"H0 = {H0} = {H0_kpc:.6f} (km/s)/kpc")
    print(f"a0 = {a0:.6f} (km/s)²/kpc")
    print(f"G = {G:.6f} kpc·(km/s)²/M_sun")
    print(f"For v200 = {v200_test} km/s: r200 = {r200_test:.1f} kpc (should be ~200-300 kpc)")

def load_sparc_data():
    """Loading SPARC data with validation"""
    url = "http://astroweb.cwru.edu/SPARC/MassModels_Lelli2016c.mrt"
    print("Loading and validating SPARC data...")

    try:
        response = requests.get(url, timeout=30)
        lines = [line for line in response.text.split('\n')[34:] if line.strip()]

        galaxies = {}
        current_galaxy = None

        for line in lines:
            try:
                galaxy_id = line[:12].strip()
                if galaxy_id != current_galaxy:
                    current_galaxy = galaxy_id
                    galaxies[galaxy_id] = []

                # Data validation and conversion
                data_point = {
                    'R': max(float(line[19:25].strip()), 0.01),  # Radius (kpc) > 0
                    'Vobs': max(float(line[26:32].strip()), 0.1), # Velocity > 0
                    'e_Vobs': max(float(line[33:38].strip() or 1.0), 0.5),  # Error ≥ 0.5 km/s
                    'Vgas': max(float(line[39:45].strip()), 0),
                    'Vdisk': max(float(line[46:52].strip()), 0),
                    'Vbul': max(float(line[53:59].strip()), 0)
                }

                # Additional checks to filter outliers
                #if data_point['R'] > 100 or data_point['Vobs'] > 500:
                #    continue

                galaxies[galaxy_id].append(data_point)
            except Exception:
                continue

        # Filtering: minimum 10 points with R > 0.1 kpc
        galaxies_filtered = {}
        for name, data in galaxies.items():
            valid_points = [d for d in data if d['R'] > 0.1]
            if len(valid_points) >= 5:
                galaxies_filtered[name] = valid_points

        print(f"Loaded {len(galaxies_filtered)} galaxies from {len(galaxies)} original ones")
        return galaxies_filtered

    except Exception as e:
        print(f"Loading error: {str(e)}")
        return {}

def v_baryonic(r, md, mb, v_gas, v_disk, v_bul):
    """Baryonic component of rotation curve"""
    # Ensuring physical values for mass-to-light ratios
    md = np.clip(md, 0.1, 10.0)  # M/L for disk within reasonable limits
    mb = np.clip(mb, 0.1, 10.0)  # M/L for bulge within reasonable limits

    # Quadratic addition of velocity components
    return np.sqrt(v_gas**2 + (md*v_disk)**2 + (mb*v_bul)**2)

def v_lcdm(r, v200, c, md, mb, v_gas, v_disk, v_bul):
    """ΛCDM model with NFW dark matter profile

    NFW profile formula for velocity (in classic form from literature):
    V_DM^2(r) = V_200^2 * [ln(1+x) - x/(1+x)] / [x * (ln(1+c) - c/(1+c))]
    where x = r/rs, rs = r200/c, r200 = V200/(10 * H0)
    """
    try:
        # Baryonic component
        v_bar = v_baryonic(r, md, mb, v_gas, v_disk, v_bul)

        # NFW profile parameters with physical constraints
        v200 = np.clip(v200, 50, 350)  # km/s
        c = np.clip(c, 3, 40)          # typical range for concentration

        # Critical radius, correct calculation in kpc
        # r200 = V200 / (10 * H0), where H0 in (km/s)/kpc
        r200 = v200 / (10 * H0_kpc)  # r200 in kpc
        rs = r200 / c                # scale radius in kpc

        # Dimensionless distance
        x = r / (rs + 1e-10)  # adding a small number to avoid division by zero

        # NFW profile for velocity - classic formula
        f_c = np.log(1 + c) - c / (1 + c)  # normalization factor for the profile

        # Calculation of velocity squared from dark matter
        v_dm_sq = np.zeros_like(r)
        mask = (x > 0) & np.isfinite(x)

        if np.any(mask):
            # Standard NFW formula for contribution to velocity squared
            v_dm_sq[mask] = v200**2 * (np.log(1 + x[mask]) - x[mask] / (1 + x[mask])) / (x[mask] * f_c)

        # Protection against negative values due to numerical errors
        v_dm_sq = np.maximum(v_dm_sq, 0)

        # Total velocity - quadratic addition of baryonic and dark components
        return np.sqrt(v_bar**2 + v_dm_sq)

    except Exception as e:
        print(f"Error in ΛCDM: {str(e)}")
        return np.full_like(r, np.nan)

def v_mond(r, md, mb, v_gas, v_disk, v_bul):
    """MOND model with standard interpolation function

    In MOND, the observed acceleration is related to the Newtonian one:
    a_obs = a_N * ν(a_N/a0)

    For "standard" interpolation:
    ν(y) = 0.5 * (1 + √(1 + 4/y))
    """
    try:
        # Baryonic component
        v_bar = v_baryonic(r, md, mb, v_gas, v_disk, v_bul)

        # Safe radius for acceleration calculation
        r_safe = np.maximum(r, 0.01)  # Avoid division by 0

        # Newtonian acceleration from visible matter (a_N = v_bar^2 / r)
        a_N = v_bar**2 / r_safe

        # Dimensionless parameter y = a_N/a0
        y = a_N / a0
        y_safe = np.maximum(y, 1e-10)  # Avoid division by very small numbers

        # Use "standard" interpolation function (McGaugh et al. 2016)
        # ν(y) = 0.5 * (1 + sqrt(1 + 4/y))
        nu = 0.5 * (1.0 + np.sqrt(1.0 + 4.0 / y_safe))

        # MOND acceleration: a_obs = a_N * ν(a_N/a0)
        a_obs = a_N * nu

        # Velocity from acceleration: v^2 = a * r
        v_mond = np.sqrt(a_obs * r_safe)

        return v_mond

    except Exception as e:
        print(f"Error in MOND: {str(e)}")
        return np.full_like(r, np.nan)

def v_eg(r, md, mb, v_gas, v_disk, v_bul):
    """Emergent Gravity model (Verlinde 2016)

    In Verlinde's theory, the additional acceleration:
    a_D = sqrt(a0 * G * M_bar / (6 * r^2))

    Corresponding velocity:
    v_D^2 = a_D * r
    """
    try:
        # Baryonic component
        v_bar = v_baryonic(r, md, mb, v_gas, v_disk, v_bul)

        # Safe radius for acceleration calculation
        r_safe = np.maximum(r, 0.01)  # Avoid division by 0

        # Estimation of baryonic mass at radius r (M_bar ∝ r * v_bar^2 / G)
        M_bar_prop = r_safe * v_bar**2 / G

        # Calculate acceleration from Emergent Gravity
        # a_D = sqrt(a0 * G * M_bar / (6 * r^2))
        a_D = np.sqrt(a0 * G * M_bar_prop / (6.0 * r_safe**2))

        # Calculate contribution to velocity squared: v_D^2 = a_D * r
        v_de_sq = a_D * r_safe

        # Total velocity - quadratic addition of baryonic and DE components
        return np.sqrt(v_bar**2 + v_de_sq)

    except Exception as e:
        print(f"Error in EG: {str(e)}")
        return np.full_like(r, np.nan)

def v_dimensional_flow(r, md, mb, r0, v_gas, v_disk, v_bul):
    """Dimensional Flow model with variable space dimensionality

    In this model, the effective space dimension d_eff = 3 - δ(r) changes with distance,
    where δ(r) is the dimension deficit:
    δ(r) = δ_0 * (1 + (r_0/r)^β)^(-1)

    For galactic rotation curves, we use spatial dependence of parameters.
    According to the paper, δ₀ = 0.71, and β ≈ 0.4-0.5 (calculated based on α = 0.95)

    Acceleration in this model:
    a(r) = G_eff(r) * M(r) / r^(2-δ(r))
    """
    try:
        # Stricter constraints on parameters for stability
        md = np.clip(md, 0.1, 10.0)  # M/L for disk
        mb = np.clip(mb, 0.1, 10.0)  # M/L for bulge
        r0 = np.clip(r0, 0.1, 200.0)  # Characteristic transition radius in kpc

        # Fixed model parameters according to the paper
        delta0 = 0.71  # Maximum dimension deficit from the paper
        beta = 0.45    # β ≈ 0.4-0.5 according to the relation with α = 0.95
        #delta0 = np.clip(delta0, 0.1, 0.99)  # Maximum dimension deficit (should be < 1)
        #beta = np.clip(beta, 0.1, 2.0)  # Power exponent

        # Safe handling of radii to avoid division by zero
        r_safe = np.maximum(r, 0.01)  # Minimum radius 0.01 kpc

        # Safe calculation of dimension deficit
        ratio = r0 / r_safe
        # Limit the ratio r0/r to avoid overflow
        ratio = np.clip(ratio, 0.0, 1.0e4)

        # Calculate dimension deficit with additional checks
        try:
            power_term = ratio**beta
            # Limit the power term for numerical stability
            power_term = np.clip(power_term, 0.0, 1.0e6)
            delta_r = delta0 / (1.0 + power_term)
        except:
            # In case of calculation problems, use approximation
            delta_r = np.where(r_safe > r0, delta0, delta0 * r_safe / r0)

        # Additionally limit delta values for stability
        delta_r = np.clip(delta_r, 0.0, 0.99)

        # Baryonic velocity with protection against negative values
        v_gas_safe = np.maximum(v_gas, 0.0)
        v_disk_safe = np.maximum(v_disk, 0.0)
        v_bul_safe = np.maximum(v_bul, 0.0)

        v_bar_sq = v_gas_safe**2 + (md*v_disk_safe)**2 + (mb*v_bul_safe)**2
        v_bar = np.sqrt(v_bar_sq)

        # Modifying factor for velocity (safe power calculation)
        delta_r_half = delta_r / 2.0
        if np.any(delta_r_half > 0.0):
            # Safe power calculation
            power_factor = np.ones_like(r_safe)
            nonzero_mask = delta_r_half > 0.0
            if np.any(nonzero_mask):
                # Use logarithm for more stable calculation
                log_r = np.log(r_safe[nonzero_mask])
                power_factor[nonzero_mask] = np.exp(delta_r_half[nonzero_mask] * log_r)
            return v_bar * power_factor
        else:
            # If deficit is zero, return just baryonic velocity
            return v_bar

    except Exception as e:
        print(f"Error in Dimensional Flow: {str(e)}")
        return np.full_like(r, np.nan)

def robust_fit(model_func, r, v_obs, v_err, v_gas, v_disk, v_bul):
    """Robust model fitting with improved optimization parameters"""

    # Number of parameters for different models
    if model_func == v_lcdm:
        n_params = 4
        bounds = [(50, 450), (1, 60), (0.1, 5.0), (0.1, 5.0)]
        x0 = [200, 10, 0.7, 0.7]
    elif model_func == v_dimensional_flow:
        n_params = 3
        # IMPORTANT CHANGE: Extended bounds for DF
        bounds = [(0.1, 10.0), (0.1, 10.0), (0.1, 200.0)]
        x0 = [0.7, 0.7, 2.0]
    else:
        n_params = 2
        bounds = [(0.2, 5.0), (0.2, 5.0)]
        x0 = [0.7, 0.7]

    def chi2_func(params):
        v_pred = model_func(r, *params, v_gas, v_disk, v_bul)
        if np.any(np.isnan(v_pred)) or np.any(np.isinf(v_pred)):
            return np.inf
        errors = np.maximum(v_err, 2.0)
        chi2_val = np.sum(((v_obs - v_pred) / errors)**2)
        return chi2_val

    # Using global optimization for DF and ΛCDM
    if model_func in [v_lcdm, v_dimensional_flow]:
        result = differential_evolution(
            chi2_func, bounds, maxiter=300, popsize=30, tol=0.01,
            mutation=(0.5, 0.9), recombination=0.8, polish=True
        )
        if result.success and result.fun < np.inf:
            return result.fun, n_params, result.x

    # Local methods
    methods = ['Nelder-Mead', 'Powell', 'SLSQP', 'L-BFGS-B']
    best_result = None

    for method in methods:
        result = minimize(chi2_func, x0, method=method, bounds=bounds,
                          options={'maxiter':5000, 'xatol':1e-8, 'ftol':1e-8})
        if result.success and (best_result is None or result.fun < best_result.fun):
            best_result = result

    # If local methods didn't work, try global again
    if best_result is None or not best_result.success:
        result = differential_evolution(
            chi2_func, bounds, maxiter=300, popsize=30, tol=0.01,
            mutation=(0.5, 0.9), recombination=0.8, polish=True
        )
        if result.success:
            best_result = result

    if best_result is not None and best_result.success:
        return best_result.fun, n_params, best_result.x

    return np.nan, n_params, np.array([np.nan]*n_params)

def calc_metrics(chi2, n_params, n_points):
    """Calculation of statistical metrics for model comparison"""
    if np.isnan(chi2) or n_points <= n_params:
        return np.nan, np.nan, np.nan

    # Degrees of freedom
    dof = n_points - n_params

    # Reduced χ² (should be ~1 for a good model)
    chi2_red = chi2 / dof

    # Akaike Information Criterion (AIC)
    # AIC = χ² + 2k, where k is the number of parameters
    aic = chi2 + 2 * n_params

    # Bayesian Information Criterion (BIC)
    # BIC = χ² + k*ln(n), where n is the number of points
    bic = chi2 + n_params * np.log(n_points)

    return chi2_red, aic, bic

def analyze_galaxy(galaxy_name, galaxy_data):
    """Complete analysis of one galaxy"""
    try:
        # Data preparation
        r = np.array([d['R'] for d in galaxy_data])
        v_obs = np.array([d['Vobs'] for d in galaxy_data])
        v_err = np.array([d['e_Vobs'] for d in galaxy_data])
        v_gas = np.array([d['Vgas'] for d in galaxy_data])
        v_disk = np.array([d['Vdisk'] for d in galaxy_data])
        v_bul = np.array([d['Vbul'] for d in galaxy_data])

        n_points = len(r)

        # Model fitting
        lcdm_chi2, lcdm_nparams, lcdm_params = robust_fit(v_lcdm, r, v_obs, v_err, v_gas, v_disk, v_bul)
        mond_chi2, mond_nparams, mond_params = robust_fit(v_mond, r, v_obs, v_err, v_gas, v_disk, v_bul)
        eg_chi2, eg_nparams, eg_params = robust_fit(v_eg, r, v_obs, v_err, v_gas, v_disk, v_bul)
        df_chi2, df_nparams, df_params = robust_fit(v_dimensional_flow, r, v_obs, v_err, v_gas, v_disk, v_bul)

        # Calculation of comparison metrics
        lcdm_metrics = calc_metrics(lcdm_chi2, lcdm_nparams, n_points)
        mond_metrics = calc_metrics(mond_chi2, mond_nparams, n_points)
        eg_metrics = calc_metrics(eg_chi2, eg_nparams, n_points)
        df_metrics = calc_metrics(df_chi2, df_nparams, n_points)

        # Optimal parameter values for verification
        # Check that parameters have reasonable values and don't hit boundaries
        if not np.isnan(lcdm_params[0]) and len(lcdm_params) > 1:
            v200 = lcdm_params[0]
            c = lcdm_params[1]
            # Calculate characteristic scales for verification
            r200 = v200 / (10 * H0_kpc)
            rs = r200 / c
            print(f"{galaxy_name}: v200={v200:.1f} km/s, c={c:.1f}, r200={r200:.1f} kpc, rs={rs:.1f} kpc")

        # Also output Dimensional Flow model parameters
        if not np.isnan(df_params[0]) and len(df_params) >= 3:
            md_df = df_params[0]
            mb_df = df_params[1]
            r0 = df_params[2]
            # Fixed model parameters
            delta0 = 0.71  # Maximum dimension deficit from the paper
            beta = 0.45    # β ≈ 0.4-0.5 according to the paper
            print(f"{galaxy_name} (DF): md={md_df:.2f}, mb={mb_df:.2f}, r0={r0:.2f} kpc, delta0={delta0:.2f}, beta={beta:.2f}")

        # Collecting results
        result = {
            'Galaxy': galaxy_name,
            'n_points': n_points,
            # Model parameters
            'LCDM_v200': lcdm_params[0] if not np.isnan(lcdm_params[0]) else np.nan,
            'LCDM_c': lcdm_params[1] if len(lcdm_params) > 1 and not np.isnan(lcdm_params[1]) else np.nan,
            'LCDM_md': lcdm_params[2] if len(lcdm_params) > 2 and not np.isnan(lcdm_params[2]) else np.nan,
            'LCDM_mb': lcdm_params[3] if len(lcdm_params) > 3 and not np.isnan(lcdm_params[3]) else np.nan,
            'MOND_md': mond_params[0] if not np.isnan(mond_params[0]) else np.nan,
            'MOND_mb': mond_params[1] if len(mond_params) > 1 and not np.isnan(mond_params[1]) else np.nan,
            'EG_md': eg_params[0] if not np.isnan(eg_params[0]) else np.nan,
            'EG_mb': eg_params[1] if len(eg_params) > 1 and not np.isnan(eg_params[1]) else np.nan,
            'DF_md': df_params[0] if not np.isnan(df_params[0]) else np.nan,
            'DF_mb': df_params[1] if len(df_params) > 1 and not np.isnan(df_params[1]) else np.nan,
            'DF_r0': df_params[2] if len(df_params) > 2 and not np.isnan(df_params[2]) else np.nan,
            # Metrics
            'LCDM_chi2': lcdm_metrics[0],  # reduced χ²
            'MOND_chi2': mond_metrics[0],
            'EG_chi2': eg_metrics[0],
            'DF_chi2': df_metrics[0],
            'LCDM_AIC': lcdm_metrics[1],
            'MOND_AIC': mond_metrics[1],
            'EG_AIC': eg_metrics[1],
            'DF_AIC': df_metrics[1],
            'LCDM_BIC': lcdm_metrics[2],
            'MOND_BIC': mond_metrics[2],
            'EG_BIC': eg_metrics[2],
            'DF_BIC': df_metrics[2],
            # Also save raw χ²
            'LCDM_raw_chi2': lcdm_chi2,
            'MOND_raw_chi2': mond_chi2,
            'EG_raw_chi2': eg_chi2,
            'DF_raw_chi2': df_chi2
        }

        return result

    except Exception as e:
        print(f"Error analyzing galaxy {galaxy_name}: {str(e)}")
        return None

def main():
    """Main galaxy analysis function"""
    # Check correctness of units
    check_units()

    # Data loading
    galaxies = load_sparc_data()
    if not galaxies:
        print("Failed to load data")
        return None, None

    # Analysis of all galaxies
    print(f"Total galaxies loaded: {len(galaxies)}")
    sample = list(galaxies.items())
    results = []

    # Analysis with progress display
    for galaxy_name, galaxy_data in tqdm(sample, desc="Analyzing galaxies"):
        result = analyze_galaxy(galaxy_name, galaxy_data)
        if result:
            results.append(result)

    # If no results
    if not results:
        print("No results for analysis")
        return None, None

    # Create DataFrame
    df = pd.DataFrame(results)

    # Statistics on NaN values
    print(f"Total results obtained: {len(df)}")
    print(f"Number of NaN values: LCDM={df['LCDM_chi2'].isna().sum()}, "
          f"MOND={df['MOND_chi2'].isna().sum()}, EG={df['EG_chi2'].isna().sum()}, "
          f"DF={df['DF_chi2'].isna().sum()}")

    # Determine best model by χ²
    df['best_model_chi2'] = df.apply(
        lambda row: 'ΛCDM' if row['LCDM_chi2'] == min(
            row['LCDM_chi2'] if not pd.isna(row['LCDM_chi2']) else np.inf,
            row['MOND_chi2'] if not pd.isna(row['MOND_chi2']) else np.inf,
            row['EG_chi2'] if not pd.isna(row['EG_chi2']) else np.inf,
            row['DF_chi2'] if not pd.isna(row['DF_chi2']) else np.inf
        ) and not pd.isna(row['LCDM_chi2']) else (
            'MOND' if row['MOND_chi2'] == min(
                row['LCDM_chi2'] if not pd.isna(row['LCDM_chi2']) else np.inf,
                row['MOND_chi2'] if not pd.isna(row['MOND_chi2']) else np.inf,
                row['EG_chi2'] if not pd.isna(row['EG_chi2']) else np.inf,
                row['DF_chi2'] if not pd.isna(row['DF_chi2']) else np.inf
            ) and not pd.isna(row['MOND_chi2']) else (
                'EG' if row['EG_chi2'] == min(
                    row['LCDM_chi2'] if not pd.isna(row['LCDM_chi2']) else np.inf,
                    row['MOND_chi2'] if not pd.isna(row['MOND_chi2']) else np.inf,
                    row['EG_chi2'] if not pd.isna(row['EG_chi2']) else np.inf,
                    row['DF_chi2'] if not pd.isna(row['DF_chi2']) else np.inf
                ) and not pd.isna(row['EG_chi2']) else 'DF'
            )
        ), axis=1
    )

    # Determine best model by AIC (accounts for model complexity)
    df['best_model_aic'] = df.apply(
        lambda row: 'ΛCDM' if row['LCDM_AIC'] == min(
            row['LCDM_AIC'] if not pd.isna(row['LCDM_AIC']) else np.inf,
            row['MOND_AIC'] if not pd.isna(row['MOND_AIC']) else np.inf,
            row['EG_AIC'] if not pd.isna(row['EG_AIC']) else np.inf,
            row['DF_AIC'] if not pd.isna(row['DF_AIC']) else np.inf
        ) and not pd.isna(row['LCDM_AIC']) else (
            'MOND' if row['MOND_AIC'] == min(
                row['LCDM_AIC'] if not pd.isna(row['LCDM_AIC']) else np.inf,
                row['MOND_AIC'] if not pd.isna(row['MOND_AIC']) else np.inf,
                row['EG_AIC'] if not pd.isna(row['EG_AIC']) else np.inf,
                row['DF_AIC'] if not pd.isna(row['DF_AIC']) else np.inf
            ) and not pd.isna(row['MOND_AIC']) else (
                'EG' if row['EG_AIC'] == min(
                    row['LCDM_AIC'] if not pd.isna(row['LCDM_AIC']) else np.inf,
                    row['MOND_AIC'] if not pd.isna(row['MOND_AIC']) else np.inf,
                    row['EG_AIC'] if not pd.isna(row['EG_AIC']) else np.inf,
                    row['DF_AIC'] if not pd.isna(row['DF_AIC']) else np.inf
                ) and not pd.isna(row['EG_AIC']) else 'DF'
            )
        ), axis=1
    )

    # Output results for individual galaxies
    df['min_chi2'] = df[['LCDM_chi2', 'MOND_chi2', 'EG_chi2', 'DF_chi2']].min(axis=1)
    df_sorted = df.sort_values('min_chi2')

    print("\nResults for individual galaxies (sorted by best χ²):")
    print(df_sorted[['Galaxy', 'n_points', 'best_model_chi2',
                     'LCDM_chi2', 'MOND_chi2', 'EG_chi2', 'DF_chi2',
                     'LCDM_AIC', 'MOND_AIC', 'EG_AIC', 'DF_AIC']].head(20).to_string(index=False))

    # Output of model parameters
    print("\nExamples of model parameters for several galaxies:")
    param_cols = ['Galaxy', 'best_model_chi2',
                  'LCDM_v200', 'LCDM_c', 'LCDM_md', 'LCDM_mb',
                  'MOND_md', 'MOND_mb',
                  'EG_md', 'EG_mb',
                  'DF_md', 'DF_mb', 'DF_r0']
    print(df_sorted[param_cols].head(10).to_string(index=False))

    # Count of best models
    chi2_counts = df['best_model_chi2'].value_counts()
    aic_counts = df['best_model_aic'].value_counts()

    print("\nDistribution of best models by χ²:")
    for model in ['ΛCDM', 'MOND', 'EG', 'DF']:
        count = chi2_counts.get(model, 0)
        print(f"{model}: {count} galaxies ({count/len(df)*100:.1f}%)")

    print("\nDistribution of best models by AIC (accounting for complexity):")
    for model in ['ΛCDM', 'MOND', 'EG', 'DF']:
        count = aic_counts.get(model, 0)
        print(f"{model}: {count} galaxies ({count/len(df)*100:.1f}%)")

    # Summary statistics
    summary = pd.DataFrame({
        'Model': ['ΛCDM', 'MOND', 'EG', 'DF'],
        'Parameters': [4, 2, 2, 3],  # Number of parameters in each model
        'Mean_χ²': [
            df['LCDM_chi2'].mean(),
            df['MOND_chi2'].mean(),
            df['EG_chi2'].mean(),
            df['DF_chi2'].mean()
        ],
        'Median_χ²': [
            df['LCDM_chi2'].median(),
            df['MOND_chi2'].median(),
            df['EG_chi2'].median(),
            df['DF_chi2'].median()
        ],
        'Mean_AIC': [
            df['LCDM_AIC'].mean(),
            df['MOND_AIC'].mean(),
            df['EG_AIC'].mean(),
            df['DF_AIC'].mean()
        ],
        'Median_AIC': [
            df['LCDM_AIC'].median(),
            df['MOND_AIC'].median(),
            df['EG_AIC'].median(),
            df['DF_AIC'].median()
        ],
        'Best_by_χ²': [
            chi2_counts.get('ΛCDM', 0),
            chi2_counts.get('MOND', 0),
            chi2_counts.get('EG', 0),
            chi2_counts.get('DF', 0)
        ],
        'Best_by_χ²(%)': [
            chi2_counts.get('ΛCDM', 0) / len(df) * 100,
            chi2_counts.get('MOND', 0) / len(df) * 100,
            chi2_counts.get('EG', 0) / len(df) * 100,
            chi2_counts.get('DF', 0) / len(df) * 100
        ],
        'Best_by_AIC': [
            aic_counts.get('ΛCDM', 0),
            aic_counts.get('MOND', 0),
            aic_counts.get('EG', 0),
            aic_counts.get('DF', 0)
        ],
        'Best_by_AIC(%)': [
            aic_counts.get('ΛCDM', 0) / len(df) * 100,
            aic_counts.get('MOND', 0) / len(df) * 100,
            aic_counts.get('EG', 0) / len(df) * 100,
            aic_counts.get('DF', 0) / len(df) * 100
        ],
        'Success_rate(%)': [
            100 * (1 - df['LCDM_chi2'].isna().mean()),
            100 * (1 - df['MOND_chi2'].isna().mean()),
            100 * (1 - df['EG_chi2'].isna().mean()),
            100 * (1 - df['DF_chi2'].isna().mean())
        ]
    })

    print("\nSummary statistics:")
    print(summary.to_string(index=False))

    # Statistical analysis of differences between models
    print("\nStatistical analysis of differences between models:")

    # Calculate AIC difference for each pair of models
    valid_mask = (~df['LCDM_AIC'].isna()) & (~df['MOND_AIC'].isna()) & (~df['EG_AIC'].isna()) & (~df['DF_AIC'].isna())

    if valid_mask.sum() > 5:  # If there is enough data for comparison
        df_valid = df[valid_mask]

        # AIC difference (positive difference means the second model is better)
        lcdm_vs_mond = df_valid['LCDM_AIC'] - df_valid['MOND_AIC']
        lcdm_vs_eg = df_valid['LCDM_AIC'] - df_valid['EG_AIC']
        lcdm_vs_df = df_valid['LCDM_AIC'] - df_valid['DF_AIC']
        mond_vs_eg = df_valid['MOND_AIC'] - df_valid['EG_AIC']
        mond_vs_df = df_valid['MOND_AIC'] - df_valid['DF_AIC']
        eg_vs_df = df_valid['EG_AIC'] - df_valid['DF_AIC']

        # Count of galaxies with significant difference (ΔAIC > 10 is considered strong evidence)
        print(f"ΛCDM significantly better than MOND: {(lcdm_vs_mond < -10).sum()} galaxies")
        print(f"MOND significantly better than ΛCDM: {(lcdm_vs_mond > 10).sum()} galaxies")
        print(f"ΛCDM significantly better than EG: {(lcdm_vs_eg < -10).sum()} galaxies")
        print(f"EG significantly better than ΛCDM: {(lcdm_vs_eg > 10).sum()} galaxies")
        print(f"ΛCDM significantly better than DF: {(lcdm_vs_df < -10).sum()} galaxies")
        print(f"DF significantly better than ΛCDM: {(lcdm_vs_df > 10).sum()} galaxies")
        print(f"MOND significantly better than EG: {(mond_vs_eg < -10).sum()} galaxies")
        print(f"EG significantly better than MOND: {(mond_vs_eg > 10).sum()} galaxies")
        print(f"MOND significantly better than DF: {(mond_vs_df < -10).sum()} galaxies")
        print(f"DF significantly better than MOND: {(mond_vs_df > 10).sum()} galaxies")
        print(f"EG significantly better than DF: {(eg_vs_df < -10).sum()} galaxies")
        print(f"DF significantly better than EG: {(eg_vs_df > 10).sum()} galaxies")

        # Mean AIC difference
        print(f"\nMean AIC difference:")
        print(f"ΛCDM vs MOND: {lcdm_vs_mond.mean():.2f}")
        print(f"ΛCDM vs EG: {lcdm_vs_eg.mean():.2f}")
        print(f"ΛCDM vs DF: {lcdm_vs_df.mean():.2f}")
        print(f"MOND vs EG: {mond_vs_eg.mean():.2f}")
        print(f"MOND vs DF: {mond_vs_df.mean():.2f}")
        print(f"EG vs DF: {eg_vs_df.mean():.2f}")

    return df, summary

if __name__ == "__main__":
    df, summary = main()

Checking units of measurement:
H0 = 70.0 km / (Mpc s) = 0.070000 (km/s)/kpc
a0 = 3702.813098 (km/s)²/kpc
G = 0.000004 kpc·(km/s)²/M_sun
For v200 = 200.0 km/s: r200 = 285.7 kpc (should be ~200-300 kpc)
Loading and validating SPARC data...
Loaded 170 galaxies from 174 original ones
Total galaxies loaded: 170


Analyzing galaxies:   0%|          | 0/170 [00:00<?, ?it/s]<ipython-input-1-a34ea6248220>:319: OptimizeWarning: Unknown solver options: ftol
  result = minimize(chi2_func, x0, method=method, bounds=bounds,
<ipython-input-1-a34ea6248220>:319: OptimizeWarning: Unknown solver options: xatol
  result = minimize(chi2_func, x0, method=method, bounds=bounds,
Analyzing galaxies:   1%|          | 1/170 [00:05<16:24,  5.82s/it]

D564-8: v200=126.7 km/s, c=1.9, r200=180.9 kpc, rs=95.5 kpc
D564-8 (DF): md=1.90, mb=6.30, r0=0.10 kpc, delta0=0.71, beta=0.45


Analyzing galaxies:   1%|          | 2/170 [00:10<14:01,  5.01s/it]

D631-7: v200=285.8 km/s, c=1.2, r200=408.3 kpc, rs=348.3 kpc
D631-7 (DF): md=1.38, mb=5.73, r0=0.10 kpc, delta0=0.71, beta=0.45


...

UGCA281: v200=312.1 km/s, c=2.6, r200=445.9 kpc, rs=168.3 kpc
UGCA281 (DF): md=1.61, mb=5.21, r0=0.14 kpc, delta0=0.71, beta=0.45


Analyzing galaxies:  99%|█████████▉| 169/170 [02:57<00:00,  1.09it/s]

UGCA442: v200=275.3 km/s, c=7.2, r200=393.3 kpc, rs=54.4 kpc
UGCA442 (DF): md=2.28, mb=9.31, r0=0.10 kpc, delta0=0.71, beta=0.45


Analyzing galaxies: 100%|██████████| 170/170 [02:58<00:00,  1.05s/it]

UGCA444: v200=262.6 km/s, c=2.6, r200=375.2 kpc, rs=145.6 kpc
UGCA444 (DF): md=4.59, mb=6.36, r0=1.04 kpc, delta0=0.71, beta=0.45
Total results obtained: 170
Number of NaN values: LCDM=0, MOND=0, EG=0, DF=0

Results for individual galaxies (sorted by best χ²):
  Galaxy  n_points best_model_chi2  LCDM_chi2  MOND_chi2  EG_chi2  DF_chi2  LCDM_AIC   MOND_AIC    EG_AIC   DF_AIC
UGC07866         7              EG   0.055721   1.240676 0.052725 0.122499  8.167163  10.203382  4.263624 6.489996
 F563-V2        10              DF   0.461159   0.394119 0.301313 0.054124 10.766951   7.152956  6.410508 6.378866
UGC05829        11              EG   0.103792   2.834277 0.060726 0.129318  8.726541  29.508490  4.546531 7.034540
 NGC5005        18            MOND   0.296693   0.060764 0.173439 0.075588 12.153699   4.972216  6.775018 7.133825
 UGCA444        36            ΛCDM   0.063428   1.938528 0.187090 0.114975 10.029683  69.909946 10.361067 9.794186
UGC05414         6            MOND   1.468336   0